# API REST Eigen Example using C++

In the following demo, we will be showing the basics of API REST protocols by means of a simple library we have created. This library basically contains two elements in its C++ version:

* A **server** which basically implements an API REST interface to communicate with an installed library within it, in our case the Eigen library. This API REST interface basically exposes certain functionalities such as adding and multiplying Eigen::VectorXd and Eigen::MatrixXd in a simple way.

* A **client** in charge of easing the interaction with the server by means of API REST interface specific methods. By using the client library, one does not need to know the specifics of the API REST interface. 

So, let's get started by importing our client!

**WARNING!!** - This demo is a C++ demo! In order of being capable of running "interpretable" C++ code, some additional libraries were required to be installed using conda. More details can be found in the Dockerfile of this client and in the docs of the project. Do not expect to just copy this demo and run, since it will not work.

In [1]:
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")

In [2]:
#pragma cling load("libapi_eigen_example_rest_client")

In [3]:
// Let us create a Client object first. This starts by importing the library
#include <apieigen/rest/EigenClient.hpp>

This ``EigenClient`` class is basically the one which will handle the API REST interface with the server, together with the connection itself, the formatting of the request and so on. When constructing the class we must provide as inputs the ``baseUrl`` of the server, which is basically the endpoint we are accessing. For this demo we are running, since we deployed both client and server in a docker-compose task, these containers will recognize each other by their docker container names, since they are in the same docker network. Thus, we will provide us arguments:

* Base URL: http://cpp-rest_server_1/18080

The server is exposed by **IP 0.0.0.0** and **port 18080** as per defined in the Dockerfile of the server. Thus, the previous inputs should be provided. This ``EigenClient`` class also allows for Basic Authentication in case the server were to be protected. Please, see below how to provide the ``user`` and ``pwd`` in the following example.

This ``EigenClient`` also allows for a specific timeout for the connection, which would also have to be provided.

In [4]:
// Let us instantiate our client
auto client = ansys::rest::client::EigenClient("http:/cpp-rest_server_1:18080");

EigenClient object created.


In [5]:
// Let us instantiate our client (with BasicAuth + timeout)
auto client_ba = ansys::rest::client::EigenClient("http:/cpp-rest_server_1:18080", "myUser", "myPwd", 10);

EigenClient object created.


Setting BasicAuthentication credentials.


One initial way to see if everything is working properly, is making use of the ``request_greeting`` method available in the client, which performs a simple handshake with the server. The server will receive the request and respond to it. Let's go ahead and do it!

In [6]:
// Now, let us check if the connection to the server is adequate by requesting a greeting
client.request_greeting();

Response: Code - 200 ; Body: Greetings from the REST Eigen Server (implemented in C++)!


Now, we will proceed to perform a simple operation such as adding two ``std::vector<double>``. However, let us suppose that we were not able to do it in the client side. So let us ask the server to do it for us.... using REST!

In [7]:
// Once the connection has been checked, let us start declaring some vectors

#include <vector>

std::vector<double> vec1 = {2.0, 3.0, 4.0, 5.0};
std::vector<double> vec2 = {3.0, 7.0, 2.0, 1.0};

Now, we will call the client method ``add_vectors(...)``, and we will explain the typical process of all interface methods (``add_XXXXX(...)`` ` and ``multiply_XXXXX(...)``):

* The client performs some sanity checks to confirm that the inputs provided are as expected. This Demo has some limitations such as: only 1D, 2D std::vector containers are allowed; they must be of type double. However, if we were to interact directly with the server, much more considerations would have to be taken into account (regarding the format of the requests - that is the reason why we use a client library).

* Once the client has checked that everything is fine to go, we POST our RESOURCES. Here is where the REST world starts:

    * Servers expose resources, also known as entities, which are well understood if we compare them to objects. For example, if we want to POST a RESOURCE we should basically:

        * ``POST`` to ``${server-uri}/${resource}`` --> In our demo: ``http://cpp-rest_server_1:18080/Vectors``.

        * Our Demo server has two resources implemented ``Matrices`` and ``Vectors``. Hence, we can only handle 1D and 2D std::vector containers.

    * When POSTing a resource, the request body contains the resource's information (i.e. our request will contain the 1D std::vector we want to POST). This information is usually serialized into a JSON format and the expected keys must be known by the client and the server to allow a proper interfacing. In a real REST application, the server usually exposes its metadata, which is basically the schema/structure of the different entities implemented (their names, attributes etc). This way, end users know how to interact with the server without knowing the specifics of the implementation.

        * Imagine we were to use CURL commands, the POST request for our first Vector would look something like this:

        * ``curl -X POST "http://cpp-rest_server_1:18080/Vectors" -H "Content-Type: application/json" -d '{"value":[5.0, 23.0, 3.0, 4.0]}'``

    * If the POST was successful, we will receive an HTTP Response which will contain in its body the ID of the posted resource.

        * ``{"vector" : {"id" : 1235412 }}``

    * The server contains a DB in which the resources posted are stored/retrieved.

* After the POSTings are performed, we will now proceed to ask the server for a certain operation involving the resources submitted.

    * Servers can also admit "operations". A typical standard for defining operation-endpoints would be:

        * ``GET`` to ``${server-uri}/${operation}/${resource}`` --> In our demo: ``http://cpp-rest_server_1:18080/add/Vectors``

    * These GET requests will also contain in its body the IDs of the involved resources:

        * Imagine we were to use CURL commands, the GET request for adding two Vectors would look something like this:

        * ``curl -X GET "http://http://cpp-rest_server_1:18080/add/Vectors" -H "Content-Type: application/json" -d '{"id1":1, "id2":2}'``

    * Requesting "operations" or "actions" can be done in multiple ways. In fact, out client implements yet another way of interacting with the server:

        * ``GET`` to ``${server-uri}/${operation}/${resource}/${resourceID_1}/${resourceID_2}`` --> In our demo: ``http://cpp-rest_server_1:18080/add/Vectors/1/2``

    * The server will then interact with the DB, retrieve the Vectors, call the Eigen Library, perform the operation and return the result.

    * The client will then receive a response containing the result of the operation:

        * ``{"vector-addition" : {"result" : [2.0, 3.0, 5.0, 4.0] }}``

        * The dedicated client implemented will then parse this JSON string and transform the resulting value into a std::vector container.

        * This way, the end user calls the client library with std::vector containers and retrieves std::vector containers, without having to know the specifics of the interface.


Let us now call the method!

In [8]:
// Let us add the vectors!
client.add_vectors(vec1, vec2)

Request: POST /Vectors Content: {"value":[2.0,3.0,4.0,5.0]}



Response: Code - 201 ; Body: {"vector":{"id":1}}


Request: POST /Vectors Content: {"value":[3.0,7.0,2.0,1.0]}



Response: Code - 201 ; Body: {"vector":{"id":2}}


Request: GET /add/Vectors/1/2


Response: Code - 200 ; Body: {"vector-addition":{"result":"[5.000000, 10.000000, 6.000000, 6.000000]"}}


{}

As mentioned before, there are several other methods implemented:

In [9]:
// Let us compute the dot product of the vectors!
client.multiply_vectors(vec1, vec2)

Request: POST /Vectors Content: {"value":[2.0,3.0,4.0,5.0]}



Response: Code - 201 ; Body: {"vector":{"id":3}}


Request: POST /Vectors Content: {"value":[3.0,7.0,2.0,1.0]}



Response: Code - 201 ; Body: {"vector":{"id":4}}


Request: GET /multiply/Vectors/3/4


Response: Code - 200 ; Body: {"vector-multiplication":{"result":40.0}}


40.000000

Let us show as well operations with 2D std::vector containers (i.e. Matrices).

In [10]:
// Now, let us do the same for matrices!
std::vector<std::vector<double>> mat1 = {{2.0, 3.0}, {4.0, 5.0}};
std::vector<std::vector<double>> mat2 = {{3.0, 7.0}, {2.0, 1.0}};

In [11]:
// Let us add the matrices!
client.add_matrices(mat1, mat2)

Request: POST /Matrices Content: {"value":[[2.0,3.0],[4.0,5.0]]}



Response: Code - 201 ; Body: {"matrix":{"id":5}}


Request: POST /Matrices Content: {"value":[[3.0,7.0],[2.0,1.0]]}



Response: Code - 201 ; Body: {"matrix":{"id":6}}


Request: GET /add/Matrices/5/6


Response: Code - 200 ; Body: {"matrix-addition":{"result":"[[5.000000, 10.000000], [6.000000, 6.000000]]"}}


{}

In [12]:
// Let us multiply the matrices!
client.multiply_matrices(mat1, mat2)

Request: POST /Matrices Content: {"value":[[2.0,3.0],[4.0,5.0]]}



Response: Code - 201 ; Body: {"matrix":{"id":7}}


Request: POST /Matrices Content: {"value":[[3.0,7.0],[2.0,1.0]]}



Response: Code - 201 ; Body: {"matrix":{"id":8}}


Request: GET /multiply/Matrices/7/8


Response: Code - 200 ; Body: {"matrix-multiplication":{"result":"[[12.000000, 17.000000], [22.000000, 33.000000]]"}}


{}